In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib import pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/multistage-continuousflow-manufacturing-process/continuous_factory_process.csv')

In [ ]:
stage2cols = [col for col in df if col.startswith('Stage2')]
machine4_cols = [col for col in df if col.startswith('Machine4')]
machine5_cols = [col for col in df if col.startswith('Machine5')]
df.drop(columns=stage2cols+machine4_cols+machine5_cols, inplace=True)

In [ ]:
setpoint_cols = [col for col in df if col.endswith('Setpoint')]
df_setpoin = df[setpoint_cols]
df.drop(columns=setpoint_cols, inplace=True)

In [ ]:
machine3_cols = [col for col in df if col.startswith('Machine3')]
df[machine3_cols]

In [ ]:
fig = px.line(df, y=machine3_cols)
fig.show()

## Variaveis da maquina 1

- PCA no material property
- Raw Material Feeder com varios outliers
- Suavizar as variaveis de motor, pressure, 

## Duvidas
- Dropar Zone 1, 2 and exit temperature?

In [ ]:
# Raw material Feeder
max_lim = df['Machine3.RawMaterialFeederParameter.U.Actual'].mean() + (3*df['Machine3.RawMaterialFeederParameter.U.Actual'].std())
min_lim = df['Machine3.RawMaterialFeederParameter.U.Actual'].mean() - (3*df['Machine3.RawMaterialFeederParameter.U.Actual'].std())
df['Machine3.RawMaterialFeederParameter.U.Actual'].clip(lower=min_lim, upper=max_lim, inplace=True)


In [ ]:
def smooth_curve(col):
    result = seasonal_decompose(col, model='additive', period=100, two_sided=False)
    result.trend.fillna(df[col].rolling(5).mean(), inplace=True)
    return result.trend.fillna(method='bfill')


In [ ]:
cols_to_smooth = ['Machine3.RawMaterialFeederParameter.U.Actual',
                  'Machine3.MotorAmperage.U.Actual',
                  'Machine3.MotorRPM.C.Actual',
                  'Machine3.MaterialPressure.U.Actual']


In [ ]:
df[cols_to_smooth] = df[cols_to_smooth].apply(smooth_curve, axis=0)

In [ ]:
result = seasonal_decompose(df['Machine3.RawMaterialFeederParameter.U.Actual'], model='additive', period=100, two_sided=False)
plt.rcParams['figure.figsize'] = (18,7)
result.plot()
pyplot.show()

In [ ]:
result.trend.fillna(method='bfill')

In [ ]:
df['Machine3.RawMaterialFeederParameter.U.Actual'].rolling(5).mean()